In [ ]:
# prompt: extract the files in the project1.rar

!unrar x "/content/drive/MyDrive/Colab Notebooks/project1.rar"

In [59]:
import os
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [60]:
btech_path = "/content/btech/found/"
bpharma_path = "/content/bpharam/found/"

In [61]:
def scrape_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

        div = soup.find("div", {"id": "midd_part_UN"})
        if not div:
            return file_path, None, None

        table = div.find_all("table")[-1]
        if not table:
            return file_path, None, None

        td_all = table.findAll("td", {"class": "border1"})
        if not td_all:
            return file_path, None, None

        try:
            sgpa = td_all[0].text.replace("SGPA", "").strip()
        except IndexError:
            sgpa = None

        try:
            cgpa = td_all[2].text.replace("CGPA", "").strip()
        except IndexError:
            cgpa = None

        return file_path, cgpa, sgpa

In [62]:
with open("/content/btech/found/21021001017_1800265331.html", "r",  encoding='utf-8') as file:
  soup = BeautifulSoup(file, 'html.parser')
  div_2 = soup.find("div", {"id": "midd_part_UN"})
  table = div_2.find_all("table")[-1]
  td_all = table.findAll("td", {"class": "border1"})
  print(td_all[0].text)
  print(td_all[2].text)

SGPA  2.96 
CGPA  2.96 


In [72]:
data = []

for folder_path in [btech_path, bpharma_path]:
    for root, _, files in os.walk(folder_path):
        for file in tqdm(files):
            if file.endswith('.html'):
                file_name = os.path.join(root, file)
                try:
                    file_name_, cgpa, sgpa = scrape_data(file_name)
                    if cgpa is not None and sgpa is not None:
                        data.append((file_name_, cgpa, sgpa))
                except Exception as e:
                    continue

100%|██████████| 32434/32434 [10:19<00:00, 52.39it/s]


In [73]:
data[:5]

[('/content/btech/found/22010305044_1800265770.html',
  '2.8333333333',
  '2.8333333333'),
 ('/content/btech/found/23010603001_1800266016.html', '4.61', '4.61'),
 ('/content/btech/found/22010205044_1800265771.html', '3.65', '3.21'),
 ('/content/btech/found/22020107022_1800265595.html', '5.22', '5.22'),
 ('/content/btech/found/21010305055_1800265395.html', '5.96', '5.96')]

In [74]:
df = pd.DataFrame(data, columns=['filename', 'cgpa', 'sgpa'])
df.head()

,filename,cgpa,sgpa
0,/content/btech/found/22010305044_1800265770.html,2.8333333333,2.8333333333
1,/content/btech/found/23010603001_1800266016.html,4.61,4.61
2,/content/btech/found/22010205044_1800265771.html,3.65,3.21
3,/content/btech/found/22020107022_1800265595.html,5.22,5.22
4,/content/btech/found/21010305055_1800265395.html,5.96,5.96


In [75]:
df.to_csv("output.csv", index=True)